In [5]:
### Modelo de nivelación de la producción para múltiples periodos ###

# Instalar a siguiente paqueteria si el codigo marca error:
!pip install pulp

import pulp

# Crear el modelo de minimización
modelo = pulp.LpProblem("Nivelacion_Produccion", pulp.LpMinimize)

# Parámetros
demanda = [400, 600, 400, 500]  # Demanda ajustada restando producción fija (10 empleados * 12)
costo_contratacion = 200
costo_despido = 400
costo_almacen = 50
p_temporal = 10  # Producción por trabajador temporal
n_periodos = 4

# Variables de decisión
x = [pulp.LpVariable(f"x{i+1}", lowBound=0) for i in range(n_periodos)]    # Temporales en el mes i
S_contrata = [pulp.LpVariable(f"S_contrata{i+1}", lowBound=0) for i in range(n_periodos)]
S_despide = [pulp.LpVariable(f"S_despide{i+1}", lowBound=0) for i in range(n_periodos)]
I = [pulp.LpVariable(f"I{i+1}", lowBound=0) for i in range(n_periodos)]

# Función objetivo: minimizar costos
modelo += (
    costo_contratacion * pulp.lpSum(S_contrata) +
    costo_despido * pulp.lpSum(S_despide) +
    costo_almacen * pulp.lpSum(I)
)

# Restricciones de inventario / demanda ajustada
modelo += p_temporal * x[0] - I[0] == demanda[0]
for i in range(1, n_periodos):
    modelo += I[i-1] + p_temporal * x[i] - I[i] == demanda[i]

# Restricciones de trabajadores (dinámica de contratación / despido)
modelo += x[0] == S_contrata[0] - S_despide[0]
for i in range(1, n_periodos):
    modelo += x[i] == x[i-1] + S_contrata[i] - S_despide[i]

# Resolver el modelo
modelo.solve()

# Resultados
print(f"Estado de la solución: {pulp.LpStatus[modelo.status]}")
print(f"Costo total mínimo: ${pulp.value(modelo.objective):,.2f}\n")

for i in range(n_periodos):
    print(f"Mes {i+1}: Temporales = {x[i].varValue:.2f}, Contrata = {S_contrata[i].varValue:.2f}, Despide = {S_despide[i].varValue:.2f}, Inventario = {I[i].varValue:.2f}")


Estado de la solución: Optimal
Costo total mínimo: $19,500.00

Mes 1: Temporales = 50.00, Contrata = 50.00, Despide = 0.00, Inventario = 100.00
Mes 2: Temporales = 50.00, Contrata = 0.00, Despide = 0.00, Inventario = 0.00
Mes 3: Temporales = 45.00, Contrata = 0.00, Despide = 5.00, Inventario = 50.00
Mes 4: Temporales = 45.00, Contrata = 0.00, Despide = 0.00, Inventario = 0.00
